In [3]:
class Node:
    def __init__(self, location, product):
        self.product = product
        self.location = location 
        self.level = 0

    def __repr__(self): 
        return  "Node(%s, %s)"%(str(self.location), str(self.product))
    
    def __eq__(self, other):
        return self.product == other.product and self.location == other.location
    
    def __hash__(self):
        return hash(str(self.product)+str(self.location))

In [6]:
class Connection:
    def __init__(self,From:Node, To:Node, conn_type):
        self.From:Node = From
        self.To:Node = To
        self.connection_type:str = conn_type    
    def __repr__(self):
        return str(self.From) 

In [14]:
from pyvis.network import Network
import pandas as pd
import logging
from pathlib import Path

class NetworkView:
    def __init__(self):
        self.net = Network(notebook=True, cdn_resources='in_line')
        self.adj_list = {}
        self.graph = {}
        self.connections = {}
        self.nodes = {}
        self.nodes_w_levels = {}
        self.history = {}
        self.output = {"Network": {"Edges": [], "Nodes": []}}

    def load_data(self):
        pass

    def set_root_nodes(self, ip_df):
        from_loc_prods = set(zip(ip_df['FromLocationId'].astype(str), 
                              ip_df["FromProductId"].astype(str)))
        to_loc_prods = set(zip(ip_df['ToLocationId'].astype(str), 
                        ip_df["ToProductId"].astype(str))) 
        root_nodes = to_loc_prods - from_loc_prods  
        root_nodes_w_levels = dict.fromkeys(root_nodes, 0)
        for i in root_nodes_w_levels:
            node = Node(i[0], i[1])
            self.history[node] = []
            self.output["Network"]["Nodes"].append({"Level": 0, "Name": str(node), "__typename": "NetworkNodeOutputType"})
            self.graph[node] = {'level': 0, 'edges': {}}

        return root_nodes
    
    def iter_child_nodes(self, ip_df, current_nodes, prev_current_nodes_df):
        tmp_paths = {}
        while True: 
            current_nodes_df = pd.DataFrame(current_nodes, columns=['ToLocationId', 'ToProductId'])
            if current_nodes_df.equals(prev_current_nodes_df):
                break
            keys = list(current_nodes_df.columns.values)
            i1 = ip_df.set_index(keys).index
            i2 = current_nodes_df.set_index(keys).index
            child_nodes_df = ip_df[i1.isin(i2)]
            child_nodes = set(zip(child_nodes_df['FromLocationId'].astype(str), 
                                    child_nodes_df["FromProductId"].astype(str)))
           
            for i in child_nodes_df.to_dict(orient='records'):
                from_node = Node(i['FromLocationId'], i['FromProductId'])
                to_node = Node(i['ToLocationId'], i['ToProductId'])
                if self.history[to_node] is not None and from_node in self.history[to_node]:
                    raise Exception("Cycle detected with source node: %s and destination node: %s" % (str(from_node), str(to_node)))
             
                source_level = int(self.graph[to_node]['level'])
                if from_node not in self.graph:
                    self.graph[from_node] = {'level': str(source_level + 1), 'edges': {}}
                else:
                    level = max(int(self.graph[to_node]['level']), int(source_level + 1))
                    self.graph[from_node]['level'] = str(level)
                conn = Connection(from_node, to_node, i['TransportationMode'])
                self.output["Network"]["Edges"].append({"Source": str(from_node), "Target": str(to_node)})
                if from_node not in self.graph[to_node]['edges']:
                    self.graph[to_node]['edges'][from_node] = []
                self.graph[to_node]['edges'][from_node].append(conn)
   
                self.history[from_node] = self.history[to_node].copy()
                self.history[from_node].append(to_node)

            current_nodes = child_nodes
            prev_current_nodes_df = current_nodes_df

    def network_view(self, xsls_src: Path):
        try:
            import time
            start_time = time.time()
            sheet_name = ['bill_of_materials', 'transportation_sourcing']
            input_sht = pd.read_excel(xsls_src, sheet_name)
            bom_sheet = input_sht['bill_of_materials']
            bom_sheet['TransportationMode'] = 'Production'
            transport_sheet = input_sht['transportation_sourcing']

            ip_df = pd.concat([bom_sheet, transport_sheet], ignore_index=True)
            ip_df['FromProductId'] = ip_df['FromProductId'].astype(str)
            ip_df['ToProductId'] = ip_df['ToProductId'].astype(str)
            ip_df['FromLocationId'] = ip_df['FromLocationId'].astype(str)
            ip_df['ToLocationId'] = ip_df['ToLocationId'].astype(str)

            root_nodes = self.set_root_nodes(ip_df)
            current_nodes = root_nodes
            current_nodes_df = pd.DataFrame(current_nodes, columns=['ToLocationId', 'ToProductId'])
            prev_current_nodes_df = pd.DataFrame()
            self.iter_child_nodes(ip_df, current_nodes, prev_current_nodes_df)

            # Add nodes and edges to the network
            for index, row in ip_df.iterrows():
                self.net.add_node(row['FromProductId'], label=row['FromProductId'])
                self.net.add_node(row['ToProductId'], label=row['ToProductId'])
                self.net.add_edge(row['FromProductId'], row['ToProductId'])

            # Display the graph
            self.net.show("network.html")

            return self.output

        except Exception as e:
            logging.error('Failed to execute endpoint', exc_info=e)
            raise e

In [ ]:
from pathlib import Path

source_with_cycle = Path("/opt/camelot/py_network_demo/data/with_cycle.xlsx")
source_without_cycle = Path("/opt/camelot/py_network_demo/data/without_cycle.xlsx")

# Instantiate the updated class and test with the cleaned dataset
nv = NetworkView()

try:
    output = nv.network_view(xsls_src=source_without_cycle)
except Exception as ex:
    print(f"Error encountered: {ex}")

# Display the graph directly in the notebook
from IPython.display import IFrame
IFrame(src='network.html', width=800, height=600)

network.html
